In [1]:
import pandas as pd
import numpy as np

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 12 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 12 days


# Data Cleanning

In [2]:
df=pd.read_csv('panl.csv')

In [3]:
df=df[['orderid','date','time','action','order_type','price','shares','balance']]

In [4]:
df.rename(columns={'orderid':'oid'},inplace=True)

In [5]:
df['date']=pd.to_datetime(df['date'])

In [6]:
df=df[~df['oid'].isin(df.loc[df['order_type']=='CANCEL-INSERT','oid'].unique())]

In [7]:
df=pd.DataFrame(df.values,index=range(len(df)),columns=df.columns)

In [8]:
df['action'].replace(to_replace='T',value='S',inplace=True)

In [9]:
df.sort(['date','time'],inplace=True)

In [10]:
df['bp']=None
df['sp']=None

# Calculation

In [11]:
def calculate_pressure(qlist,idx):
    bp=0.0
    sp=0.0
    for i in qlist:
        if df.at[i,'action']=='B':
            bp+=df.at[i,'shares']*np.exp(-1.0*(df.at[idx,'time']-df.at[i,'time'])/T)
        else:
            sp+=df.at[i,'shares']*np.exp(-1.0*(df.at[idx,'time']-df.at[i,'time'])/T)
    df.at[idx,'bp']=bp
    df.at[idx,'sp']=sp
    return

In [12]:
def remove_oid_from_qlist(qlist,oid):
    oid_list=list(df.index[df['oid']==oid])
    for i in oid_list:
        try:
            qlist.remove(i)
        except:
            pass
    return

In [ ]:
qlist=[0]
T=111.111
for i in range(1,len(df)):
    print i
    if df.at[i,'date']!=df.at[i-1,'date']:
        qlist=[i] #if date change then reinitiate qlist
    else: 
        if df.at[i,'order_type'] != 'EXECUTION':
            qlist.append(i)
        else:
            calculate_pressure(qlist,i)
        if df.at[i,'balance']==0:
            remove_oid_from_qlist(qlist,df.at[i,'oid'])